In [31]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm.notebook.tqdm as tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

## Se importan los dataframes 

In [32]:
# load all metadata
#hau nire diskan daukat SAD karpetan 2020-2021

video_games = []#
insurances = []   
#dfMergedfMeta= pd.read_csv('Cell_Phones/SamsungAppleXiaomiReviews_prueba.csv')
video_games= pd.read_csv('VG-Reviews5AndMetaElecNintSonyMic_v2.csv')
insurances = pd.read_csv('HRBlockIntuitReviewsTrainDev_vLast7.csv', low_memory=False)


In [33]:
video_games.keys()

Index(['Unnamed: 0', 'asin', 'vote', 'style', 'feature', 'similar_item',
       'brand', 'also_buy', 'category', 'title', 'main_cat', 'description',
       'reviewTime', 'overall', 'reviewerID', 'reviewerName', 'unixReviewTime',
       'reviewText', 'summary', 'City', 'State', 'Country', 'Postal Code'],
      dtype='object')

In [34]:
video_games['brand'].value_counts()

by\n    \n    Electronic Arts                        19949
by\n    \n    Nintendo                               16812
by\n    \n    Sony                                   10915
by\n    \n    Microsoft                               8810
Nintendo                                              4660
Electronic Arts                                       1780
Sony                                                  1697
Microsoft                                             1230
Sony Computer Entertainment                            438
by\n    \n    Sony Computer Entertainment America      106
Nintendo of America                                     98
Electronic Arts, Inc.                                   78
by\n    \n    Sony Computer Entertainment               74
Sony Entertainment                                      52
Microsoft Corporation                                   44
Super Nintendo Super Castlevania IV                     22
Electronc Arts                                          

In [35]:
insurances.keys()

Index(['Unnamed: 0', 'asin', 'brand', 'price', 'title', 'Unnamed: 5',
       'category', 'rank', 'description', 'main_cat', 'details', 'overall',
       'vote_y', 'verified_y', 'reviewTime_y', 'reviewerID_y',
       'reviewerName_y', 'reviewText', 'summary', 'unixReviewTime', 'State',
       'Country', 'Postal Code'],
      dtype='object')

In [36]:
insurances['brand'].value_counts()

Intuit                              72752
H&R Block                           16242
by\n    \n    Intuit                 9488
HRBB9                                 657
H&amp;R Block                         358
by\n    \n    H&R Block               195
Intuit, Inc.                           27
H&R BLCOK                              17
John Truby Blockbuster                 15
Video Blocks                            4
Administaff HRTools                     4
H & R Block                             4
Intuit Inc.                             3
Teneron/Block Financial Software        3
H&R                                     3
H&R BLOCK                               2
Intuit Inc./BlueHippo                   2
Name: brand, dtype: int64

## Se obtienen en dataframes aparte las distintas marcas

In [37]:
dfNintendo = video_games[video_games['brand'].str.contains('Nintendo',na=False)]

In [38]:
dfSony = video_games[video_games['brand'].str.contains('Sony|PlayStation|Electr',na=False)]

In [39]:
dfMicrosoft = video_games[video_games['brand'].str.contains('[M|m]icrosoft',na=False)]

In [40]:
dfIntuit = insurances[insurances['brand'].str.contains('Intuit')]

In [41]:
dfHR = insurances[~insurances['brand'].str.contains('Intuit')]

## Se separan las reviews negativas y positivas de cada dataframe

In [42]:
dfNintendoPositives=dfNintendo[dfNintendo['overall']>3]
dfNintendoNegatives=dfNintendo[dfNintendo['overall']<=3]

In [43]:
dfSonyPositives=dfSony[dfSony['overall']>3]
dfSonyNegatives=dfSony[dfSony['overall']<=3]

In [44]:
dfMicrosoftPositives=dfMicrosoft[dfMicrosoft['overall']>3]
dfMicrosoftNegatives=dfMicrosoft[dfMicrosoft['overall']<=3]

In [45]:
dfIntuitPositives = dfIntuit[dfIntuit['overall']>3]
dfIntuitNegatives = dfIntuit[dfIntuit['overall']<=3]

In [46]:
dfHRPositives = dfHR[dfHR['overall']>3]
dfHRNegatives = dfHR[dfHR['overall']<=3]

## Se obtienen las text reviews

In [47]:
nintendoPositivesDocuments = dfNintendoPositives[dfNintendoPositives['reviewText'].notna()]['reviewText'].tolist()
nintendoNegativesDocuments = dfNintendoNegatives[dfNintendoNegatives['reviewText'].notna()]['reviewText'].tolist()

In [48]:
sonyPositivesDocuments = dfSonyPositives[dfSonyPositives['reviewText'].notna()]['reviewText'].tolist()
sonyNegativesDocuments = dfSonyNegatives[dfSonyNegatives['reviewText'].notna()]['reviewText'].tolist()

In [49]:
microsoftPositivesDocuments = dfMicrosoftPositives[dfMicrosoftPositives['reviewText'].notna()]['reviewText'].tolist()
microsoftNegativesDocuments = dfMicrosoftNegatives[dfMicrosoftNegatives['reviewText'].notna()]['reviewText'].tolist()

In [50]:
intuitPositivesDocuments = dfIntuitPositives[dfIntuitPositives['reviewText'].notna()]['reviewText'].tolist()
intuitNegativesDocuments = dfIntuitNegatives[dfIntuitNegatives['reviewText'].notna()]['reviewText'].tolist()

In [23]:
HRPositivesDocuments = dfHRPositives[dfHRPositives['reviewText'].notna()]['reviewText'].tolist()
HRNegativesDocuments = dfHRNegatives[dfHRNegatives['reviewText'].notna()]['reviewText'].tolist()

## LDA

In [ ]:
def display_topics(H, W, feature_names, documents,output_name, no_top_words = 30, no_top_documents = 10):
    f = open(output_name + ".txt", "a")
    for topic_idx, topic in enumerate(H):
        s = "Topic: " + str(topic_idx) + " -->"
        print(s)
        f.write(s)
        s = ''.join([' ' +feature_names[i] + ' ' + str(round(topic[i], 5)) 
                for i in topic.argsort()[:-no_top_words - 1:-1]])
        print(s)
        f.write(s)
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        docProbArray=np.argsort(W[:,topic_idx])
        print(docProbArray)
    
        #for doc_index in top_doc_indices:
        #    print(documents[doc_index])
        f.write("\n")
    f.close()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### LDASonyNeg_29_1.9

In [ ]:
no_topics = 29

alpha = 1.9
beta = 1.9

documents = sonyNegativesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDASonyNeg_29_1.9')

### LDAMicroPos_21_1.9

In [ ]:
no_topics = 21

alpha = 1.9
beta = 1.9

documents = microsoftPositivesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDAMicroPos_21_1.9')

### LDAMicroNeg_16_1.9

In [ ]:
no_topics = 16

alpha = 1.9
beta = 1.9

documents = microsoftNegativesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDAMicroNeg_16_1.9')

## GENSIM

In [51]:
import logging
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from pprint import pprint
nltk.download('wordnet')
nltk.download('omw-1.4')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package wordnet to /home/eiros/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/eiros/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [52]:
def preproceso(docs):
    # Tokenize 
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 1] for doc in docs]

    # Remove stopwords
    stopwords = gensim.parsing.preprocessing.STOPWORDS
    docs = [[token for token in doc if not(token in stopwords)] for doc in docs]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    # Bag-of-words
    dictionary = Dictionary(docs)

    corpus = [dictionary.doc2bow(doc) for doc in docs]
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))

    return corpus , dictionary

### GensimMicroPos

In [57]:
def gen_model(num_topics,corpus, dictionary):
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    # Make an index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token

    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    return model

### GensimMicroPos

In [54]:
corpus,dictionary = preproceso(microsoftPositivesDocuments)

2022-05-21 20:53:03,335 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-05-21 20:53:03,988 : INFO : built Dictionary<25005 unique tokens: ['best', 'buy', 'came', 'card', 'compared']...> from 7715 documents (total 711667 corpus positions)
2022-05-21 20:53:03,989 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<25005 unique tokens: ['best', 'buy', 'came', 'card', 'compared']...> from 7715 documents (total 711667 corpus positions)", 'datetime': '2022-05-21T20:53:03.989787', 'gensim': '4.2.0', 'python': '3.10.4 (main, Apr  2 2022, 09:04:19) [GCC 11.2.0]', 'platform': 'Linux-5.17.5-76051705-generic-x86_64-with-glibc2.35', 'event': 'created'}


Number of unique tokens: 25005
Number of documents: 7715


In [ ]:
f = open('GensimMicroPos[10-100]' + ".txt", "a")
for i in range(10,101):
    print("Generando modelo: " + str(i))
    model_act=gen_model(i,corpus)
    top_topics = model_act.top_topics(corpus)
    f.write('Modelo con %i clusters -->' % i)
    for frec, word in top_topics[0][0]:
        f.write(" " + word + " " + str(frec) + " |")
    f.write("\n\n")
f.close()

### GensimMicroNeg

In [ ]:
corpus,dictionary = preproceso(microsoftNegativesDocuments)

In [ ]:
f = open('GensimMicroNeg[10-100]' + ".txt", "a")
for i in range(10,101):
    print("Generando modelo: " + str(i))
    model_act=gen_model(i,corpus)
    top_topics = model_act.top_topics(corpus)
    f.write('Modelo con %i clusters -->' % i)
    for frec, word in top_topics[0][0]:
        f.write(" " + word + " " + str(frec) + " |")
    f.write("\n\n")
f.close()